# Day 1 - NB2 - Simple speech recognizer with speechrecognition

In this notebook we watch 2 short python video tutorials and code a simple speech-to-command script.


- [Video 3: Speech Recognition in Python](#Video-3:-Speech-Recognition-in-Python)
- [Video 4: Speech Recognition Using Python | How Speech Recognition Works In Python | Simplilearn](#Video-4:-Speech-Recognition-Using-Python-|-How-Speech-Recognition-Works-In-Python-|-Simplilearn)
- [Voice command script](#Voice-command-script)

# Video 3: Speech Recognition in Python
**[Speech Recognition in Python](https://www.youtube.com/watch?v=9GJ6XeB-vMg)**


### To install pyaudio:

At a system level:

```
sudo apt install python3-pyaudio
```

At an environment level:

```
sudo apt install portaudio19-dev
pip install pyaudio
```

In [1]:
!pip install speechrecognition pyttsx3 

## English speech recognizer

In [2]:
import speech_recognition

In [4]:

recognizer = speech_recognition.Recognizer()
while True:
    try:
        with speech_recognition.Microphone() as mic:
            recognizer.adjust_for_ambient_noise(mic, duration=0.2)
            audio = recognizer.listen(mic)
            text = recognizer.recognize_google(audio)
            text = text.lower()
            print(f"Recognized: {text}")
    except speech_recognition.UnknownValueError:
        recognizer = speech_recognition.Recognizer()
        continue
    except KeyboardInterrupt:
        print("Exiting loop")
        break

Recognized: hello hello
Recognized: hi how are you doing
Recognized: hey siri
Recognized: speech recognition is an interdisciplinary subfield of computer science and computation of linguistics that develops methodologies and technologies that enable the recognition and translation of spoken language into text by computers with the main benefit of searchability
Recognized: okay this is snoot
Recognized: meet not snoot
Recognized: meet
Recognized: meet
Recognized: meet
Recognized: neat
Recognized: okay that's okay so i should have extended my esound
Recognized: it takes some time to recognize this audio do
Exiting loop


## Spanish: doesn't work out of the box

In [9]:
recognizer = speech_recognition.Recognizer()
while True:
    try:
        with speech_recognition.Microphone() as mic:
            recognizer.adjust_for_ambient_noise(mic, duration=0.2)
            audio = recognizer.listen(mic)
            text = recognizer.recognize_google(audio, 'es-AR')
            text = text.lower()
            print(f"Recognized: {text}")
    except speech_recognition.UnknownValueError:
        recognizer = speech_recognition.Recognizer()
        continue
    except KeyboardInterrupt:
        print("Exiting loop")
        break
    except Exception as e:
        print(e)
        break

recognition request failed: Forbidden


# Video 4: Speech Recognition Using Python | How Speech Recognition Works In Python | Simplilearn
**[Speech Recognition Using Python | How Speech Recognition Works In Python | Simplilearn](https://www.youtube.com/watch?v=PWVH3Vx3dCI)**

In [10]:
!pip install SpeechRecognition
!pip install apiai

     |████████████████████████████████| 16.8 MB 11.3 MB/s eta 0:00:01


In [11]:
import speech_recognition as sr
import pyttsx3

### TTS with `pyttsx3`

In [13]:
def speak_text(command):
    engine = pyttsx3.init()
    engine.say(command)
    engine.runAndWait()

In [14]:
speak_text("hello world")

OSError: libespeak.so.1: cannot open shared object file: No such file or directory

Trying [this](https://stackoverflow.com/questions/65274843/oserror-libespeak-so-1-cannot-open-shared-object-file-no-such-file-or-directo):
```
sudo apt install espeak
```

Works!

In [19]:
# It works but it's terrible
speak_text("hello world!")

In [22]:
engine = pyttsx3.init("espeak")
voices = engine.getProperty('voices')

In [27]:
print(voices[11].languages)

[b'\x02en-gb']


In [33]:
# https://askubuntu.com/questions/1298356/espeak-text-to-speech-sounds-very-weird-while-running-pyttsx3-code-on-ubuntu-20

def speak_text_v2(command):
    engine = pyttsx3.init("espeak")
    voices = engine.getProperty('voices')
    engine.setProperty('voice',voices[11].id) #English
    engine.say(command)
    engine.runAndWait()

# Still bad
speak_text_v2("Hello World and this is a test.")

In [40]:

def speak_text_v3(command):
    engine = pyttsx3.init("espeak")
    voices = engine.getProperty('voices')
    engine.setProperty('voice',voices[10].id) #English
    engine. setProperty("rate", 156)
    engine.say(command)
    engine.runAndWait()

speak_text_v3("Hello World and this is a test.")

### Recognize voice and output it with TTS

In [42]:
def recognize_and_speak():
    recognizer = speech_recognition.Recognizer()
    while True:
        try:
            with speech_recognition.Microphone() as mic:
                recognizer.adjust_for_ambient_noise(mic, duration=0.2)
                audio = recognizer.listen(mic)
                text = recognizer.recognize_google(audio)
                text = text.lower()
                print(f"Recognized: {text}")
                speak_text_v3(text)
        except speech_recognition.UnknownValueError:
            recognizer = speech_recognition.Recognizer()
            continue
        except KeyboardInterrupt:
            print("Exiting loop")
            break
        except Exception as e:
            print(e)
            break
            
recognize_and_speak()

Recognized: hello this is a test
Exiting loop


### Open a website

In [43]:
import webbrowser as web

In [47]:
CHROME_PATH = '/usr/bin/google-chrome %s'
url = "dataista.com.ar"
web.get(CHROME_PATH).open(url)

Opening in existing browser session.


True

# Voice command script

In [63]:
%%writefile ../src/asr_url_opener.py

import sys
import webbrowser as web
import speech_recognition as sr
CHROME_PATH = '/usr/bin/google-chrome %s'

import warnings; warnings.simplefilter('ignore')

def open_url(url):
    web.get(CHROME_PATH).open(url)

def text_to_command(text):
    print(f"Recognized: {text}")
    if text.startswith("open url"):
        url = text.replace("open url", "")
        open_url(url)
    if text == 'exit':
        print("Exiting")
        sys.exit()
        
    
def main_loop():
    r = sr.Recognizer()
    while True:
        try:
            with sr.Microphone() as mic:
                r.adjust_for_ambient_noise(mic, duration=0.2)
                audio = r.listen(mic)
                text = r.recognize_google(audio)
                text = text.lower()
                text_to_command(text)
                
        except sr.UnknownValueError:
            recognizer = sr.Recognizer()
            continue
        except KeyboardInterrupt:
            print("Exiting main loop")
            break
        except Exception as e:
            print(e)
            recognizer = sr.Recognizer()
            continue
    
if __name__ == '__main__':
    main_loop()
    

Overwriting ../src/asr_url_opener.py


In [62]:
!python ../src/asr_url_opener.py

ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
Recognized: hello this is a test
Recognized: open euro twitter.com
Recognized: open url twitter.com
Opening in existing browser session.
Recognized: open url google.com
Opening in existing br

To solve the error / warnings:
https://stackoverflow.com/questions/7088672/pyaudio-working-but-spits-out-error-messages-each-time


To run in background:

```bash
mkdir logs/
cd src/
python asr_url_opener.py > ../logs/asr_url_opener.log 2>&1 &
```

The script [scripts/asr_url_opener.sh](../scripts/asr_url_opener.sh) runs in a slight improvement of the code above:

In [64]:
!cat ../scripts/asr_url_opener.sh

#!/bin/sh
# Run from the scripts/ path as ./asr_url_opener.sh
# Make sure you are in the environment with speech_recognition installed!
python -u ../src/asr_url_opener.py  >> ../logs/asr_url_opener.log 2>&1 & 


It's actually running and working now...! :D